# Jacobian: Fake news vs. direct method

In [1]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt   
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

from HANCModel import HANCModelClass

## Setup and steady state 

In [2]:
model = HANCModelClass(name='baseline') # create an instance of the model
model.find_ss(method='direct',do_print=False)

c:\Users\fcv495\Anaconda3\lib\site-packages\numba\np\ufunc\parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [3]:
model.par.T

400

In [4]:
model_fakenews = model.copy()

# Jacobians

Direct approach 

In [5]:
model._compute_jac_hh(do_print=False, do_direct=True)

In [6]:
t0 = time.time()
model._compute_jac_hh(do_print=True, do_direct=True)
t1 = time.time()

time_direct = t1 - t0

finding Jacobian wrt. ghost          : 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267

Number of runs needed:

In [10]:
# number of inputs times number of time periods 
print(f'Number of runs: {3*model.par.T}') 

Number of runs: 1200


Fake-news

In [13]:
t0 = time.time()
model_fakenews._compute_jac_hh(do_print=True, do_direct=False) # do_direct=False is fake-news algorithm
t1 = time.time()
time_fakenews = t1 - t0

one step deviation from steady state calculated in 0.0 secs
curly_Y and curly_D calculated for r               in 0.5 secs
curly_Y and curly_D calculated for w               in 0.6 secs
curly_E calculated in 0.3 secs
builiding blocks combined in 0.2 secs
household Jacobian computed in 1.6 secs


In [14]:
# number of inputs
print(f'Number of runs: {3}') 

Number of runs: 3


In [15]:
print(f'Speed up relative to direct method: {time_direct/time_fakenews:.1f}')

Speed up relative to direct method: 161.1


Check that we get same approx. same Jacobians 

In [19]:
for key in model.jac_hh:
    max_error = np.max(model_fakenews.jac_hh[key] - model.jac_hh[key])
    print(f'Max error for {key} : {max_error:.5f}')

Max error for ('A_hh', 'r') : 0.00000
Max error for ('A_hh', 'w') : 0.00002
Max error for ('C_hh', 'r') : 0.00001
Max error for ('C_hh', 'w') : 0.00001
Max error for ('L_hh', 'r') : 0.00000
Max error for ('L_hh', 'w') : 0.00000
